##### Quantum Data Science 2021/2022
### Lecture 1 - Introduction to quantum machine learning - The swap test and the swap test classifier 

#### Import libraries

In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, Aer, execute, transpile
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

##### Function for executing a quantum circuit 

In [2]:
def execute_circuit(qc, shots=1024, device=None):
    if device is None:
        device = Aer.get_backend('qasm_simulator')
    else:
        device = device
    
    circ_trans = transpile(qc,device)
    counts = device.run(circ_trans, shots=shots).result().get_counts()
    
    return counts

#### <span style="color: red;">EXERCISE 1</span>: 
Design a function *basis_states_probs* that executes an arbitrary quantum circuit and returns the probability of each of the basis states.

Test the function by executing the quantum circuit for the Bell state $ |\psi\rangle = \frac{1}{\sqrt{2}} (|00\rangle + |11\rangle)$

In [3]:
### SOLUTION ###
 
def basis_states_probs(counts, shots=1024, n_qubits=1):
   probs = []
   basis_states = [np.binary_repr(i,width=n_qubits) for i in range(2**n_qubits)]

   for b in basis_states:
      c = counts.get(b)
      if c is None:
         probs.append(0)
      else:
         probs.append(counts[b]/shots)


   return probs

In [6]:
qr = QuantumRegister(2, "qr")
qc = QuantumCircuit(qr)

qc.h(qr[0])
qc.cx(qr[0],qr[1])

qc.measure_all()

counts = execute_circuit(qc,shots=1024)
probs = basis_states_probs(counts, shots=1024, n_qubits=2)

print("probability |00> - {}".format(probs[0]))
print("probability |01> - {}".format(probs[1]))
print("probability |10> - {}".format(probs[2]))
print("probability |11> - {}".format(probs[3]))

probability |00> - 0.4931640625
probability |01> - 0
probability |10> - 0
probability |11> - 0.5068359375


/opt/homebrew/Caskroom/miniforge/base/envs/quantum_DS/lib/python3.10/site-packages/numpy/linalg/linalg.py:2158: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/opt/homebrew/Caskroom/miniforge/base/envs/quantum_DS/lib/python3.10/site-packages/numpy/linalg/linalg.py:2158: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


#### The swap test:

<p align="center">
  <img width="400" height="200" src="images/swaptest.jpeg">
</p>

#### <span style="color: red;">EXERCISE 2</span>
Prove that the probability of measuring the ancilla in state $|0\rangle$ is given by: 
$$ P(0) = \frac{1}{2} + \frac{1}{2} |\langle \psi | \phi \rangle |^2$$ 

What should be the overlap between the states $|\psi\rangle = |+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$ and state $|\psi\rangle = |-\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$ . Compute the overlap between the states.

In [90]:
### SOLUTION ###

def one_qubit_swap_test(qc, ancilla, psi, phi, cr):

    overlap = 0 
    
    qc.h(ancilla)
    qc.cswap(ancilla, psi, phi)
    qc.h(ancilla)

    qc.measure(ancilla, cr)

    counts = execute_circuit(qc,shots=1024, device=None)
    probs = basis_states_probs(counts, shots=1024, n_qubits=1)
    p0 = probs[0]

    overlap += 2*p0 - 1
    return overlap

ancilla = QuantumRegister(1)
psi = QuantumRegister(1)
phi = QuantumRegister(1)
cr = ClassicalRegister(1)
qc = QuantumCircuit(ancilla, psi, phi, cr)

qc.h(psi)
qc.x(phi)
qc.h(phi)

overlap = one_qubit_swap_test(qc, ancilla, psi, phi, cr)

print("Overlap |<a|b>|^2 - {}".format(overlap))

Overlap |<a|b>|^2 - 0.0


### <span style="color: red;">EXERCISE 2</span> - Let's make a classifier using the swap test

Consider the problem of **binary classification** - You have a labelled dataset $D = \{ (x_1,y_1) , \dots , (x_N,y_N)\}$ where $x$ corresponds to a point and $y$ to the respective label. As the name suggests, in binary classification $y$ takes only two values. For instance, suppose that points $x$ are images of cats vs dogs. In this case, $y$ corresponds to the label "cat" or "dog". Mathematically speaking we could use "cat" $\mapsto 0$ and "dog" $\mapsto 1$.

Consider now a dataset of single qubit quantum states $D = \{ (x_1,y_1) , \dots , (x_N,y_N)\}$ where $x$ corresponds to a quantum state and $y$ to the label. $0$ in the case of the state being closer to state $|0\rangle$ and $1$ if the state is closer to state $|1\rangle$.

<p align="center">
  <img width="500" height="400" src="images/bin_quantum.jpeg"> 
</p>


The classification for each new unlabelled quantum state in $D'$ can be done by measuring the distance to every quantum state in $D$. The distance is calculated individually for each label and averaged. The label that has the smallest average distance should be the assigned label to the new quantum state in $D'$. Let $\overline{d_{i}^{0}}$ be the average distance between the new quantum state $|\phi_i\rangle$ and every quantum state of label $0$ in $D$, $|\psi_j^0\rangle$ and $\overline{d_{i}^{1}}$ be the average distance between the new quantum state $i$ and every quantum state of label $1$, $|\psi_j^1\rangle$. The distance can be calculated using the swap test presented above, where the distance is given by the overlap between two quantum states.

$$ \overline{d_{i}^{0}} = \frac{1}{N} \sum_{j=1}^{N} |\langle \psi_j^0 | \phi_i \rangle|^2$$

$$ \overline{d_{i}^{1}} = \frac{1}{N} \sum_{j=1}^{N} |\langle \psi_j^1 | \phi_i \rangle|^2$$

The label for the new quantum state is assigned dependently on the average distances. 

$$ y_i = \left\{
\begin{array}{ll}
      0 & \overline{d_{i}^{0}} \geq \overline{d_{i}^{1}} \\
      1 &  otherwise \\
\end{array} 
\right. $$

Let's start by loading the labelled dataset, $d$ and unlabelled dataset $d\_$ with quantum states that we want to classify.

Load datasets from file q_dataset.npy
    
    - d is the dataset with quantum state vector and labels [((a_1, b_1), label1) , ...]
    - d_bloch is the dataset with bloch vector representations to plot in the bloch sphere [((x_1, y_1, z_1), label1) , ...] 
    - d_ is the unlabelled dataset with only quantum state vectos [((a_1, b_1)]
    - d__bloch is the unlabelled dataset with bloch vector representations to plot in the bloch sphere [(x_1, y_1, z_1) , ...]

In [91]:
with open("q_dataset.npy", 'rb') as f:
    d = np.load(f, allow_pickle=True)
    d_bloch = np.load(f, allow_pickle=True)    
    d_ = np.load(f, allow_pickle=True)
    d__bloch = np.load(f, allow_pickle=True)

Plot the data i.e., plot the single-qubit quantum states in the bloch sphere to understand your data. Use **d_bloch**

In [92]:
from kaleidoscope import bloch_sphere

vec = []
colors = []

for i in d_bloch:
    if i[1]:
        colors.append("#AA00FF")
    else:
        colors.append("#AA00F")

    vec.append(i[0])

for i in d__bloch:
    colors.append("#00FF00")
    vec.append(i)

bloch_sphere(points=vec, points_color=colors)


0.0


##### Use the swap test to classify the quantum states present in $d\_$

You can choose to either use the euclidean distance or the original inner product distance. 

**Note:** Use the qiskit function **initialize** to prepare arbitrary quantum states from statevector

- x = QuantumRegister(1)
- x_ = QuantumRegister(1)
- qc = QuntumCircuit(x,x_)

- qc.initialize(d[0], x)
- qc.initialize(d_[0], x_)

##### 4.1 - create a function that outputs the distance between two quantum states: 

In [93]:
### SOLUTION ###

def distance(qc, ancilla, x, x_, cr):

    distance = 0

    overlap = one_qubit_swap_test(qc, ancilla, x, x_, cr)
    
    distance += overlap
    
    return distance 

##### 4.2 - create a function that outputs the average distance for elements of a given class

In [94]:
### SOLUTION ### 

def avg_distance(d, x_):

    avg_distance = np.zeros(2)

    for x in d:
        ancilla = QuantumRegister(1)
        xr = QuantumRegister(1)
        xr_ = QuantumRegister(1)
        cr = ClassicalRegister(1)
        qc = QuantumCircuit(ancilla, xr, xr_, cr)

        qc.initialize(x[0], xr)
        qc.initialize(x_, xr_)   

        overlap = distance(qc, ancilla, xr, xr_, cr)
        avg_distance[x[1]] += overlap

    avg_distance /= len(x)
    return avg_distance

#### 4.3 - Create function that classifies each quantum state in $d\_$

In [97]:
### SOLUTION ###

def classify(d,d_):

    labels = []

    for x_i in d_:

        avg_dist = avg_distance(d, x_i)

        if avg_dist[0] >= avg_dist[1]:
            labels.append(0)
        else:
            labels.append(1)

    return labels
    

#### Visualize the obtained classification by plotting again in the bloch sphere

In [99]:
vec = []
colors = []

labels = classify(d,d_)
new_d__bloch = []

for i in range(len(d__bloch)):
    new_d__bloch.append([d__bloch[i],labels[i]])

for i in new_d__bloch:
    if i[1]:
        #purple
        colors.append("#AA00FF")
    else:
        #red
        colors.append("#AA00F")

    vec.append(i[0])

bloch_sphere(points=vec, points_color=colors)


### Homework
- To classify each point we need to compare it with every single point in the dataset. 
- What if instead of computing distances between every point, we had a centroid for the label, for instance the average distance between each datapoint in the dataset? Then, to classify new points, we would only need to compare the distance to centroid instead of every point. Experiment and compare the results with the previous classifier.